In [38]:
# Dependencies and Setup
import pandas as pd
import numpy as np
# File to Load 
file = "./Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file)
purchase_data_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [2]:
#Total Number of Unique Players
total_players = len(purchase_data_df["SN"].unique())
total_players

576

In [3]:
#Check to see if there were repeat buyers
purchase_data_df["SN"].value_counts()

Lisosia93       5
Iral74          4
Idastidru52     4
Saedaiphos46    3
Tyidaim51       3
               ..
Thryallym62     1
Iskosia90       1
Rairin89        1
Sondassan80     1
Eulolis41       1
Name: SN, Length: 576, dtype: int64

In [4]:
#Create Purchase Analysis Dataframe
purchase_summary_dict = {
    
    'Number of Unique Items': [len(purchase_data_df["Item Name"].unique())],
    'Total Number of Purchases': [len(purchase_data_df)],
    'Average Purchase Price': [round(purchase_data_df["Price"].mean(),2)],
    'Total Revenue': [purchase_data_df["Price"].sum()]
    }

summary_df = pd.DataFrame(purchase_summary_dict)

In [5]:
#Format Purchase Analysis Dataframe
summary_df['Average Purchase Price'] = summary_df['Average Purchase Price'].map('${:.2f}'.format)
summary_df['Total Revenue'] = summary_df['Total Revenue'].map('${:.2f}'.format)
summary_df

,Number of Unique Items,Total Number of Purchases,Average Purchase Price,Total Revenue
0,179,780,$3.05,$2379.77


In [6]:
#Gender Demograpgics Variables - Female
f_df = purchase_data_df.loc[purchase_data_df['Gender'] == 'Female']
f_total = len(f_df['SN'].unique())
f_per = (f_total/total_players)

In [7]:
#Gender Demograpgics Variables - Non-Disclosed
nd_df = purchase_data_df.loc[purchase_data_df['Gender'] == 'Other / Non-Disclosed']
nd_total = len(nd_df['SN'].unique())
nd_per = (nd_total/total_players)

In [8]:
#Gender Demograpgics Variables - Male
m_df = purchase_data_df.loc[purchase_data_df['Gender'] == 'Male']
m_total = len(m_df['SN'].unique())
m_per = (m_total/total_players)

In [9]:
#Create Gender Demographics Dataframe
gender_summary_dict = {
    'Gender': ['Female', 'Male', 'Other/Non-Disclosed'], 
    'Total Players': [f_total, m_total, nd_total],
    'Percent of Players': [f_per, m_per, nd_per],
    }

gender_demographics_df = pd.DataFrame(gender_summary_dict)
gender_demographics_df = gender_demographics_df.set_index('Gender')
gender_demographics_df

,Total Players,Percent of Players
Gender,,
Female,81,0.140625
Male,484,0.840278
Other/Non-Disclosed,11,0.019097


In [10]:
#Apply Formatting to Columns in Gender Demographics Dataframe
gender_demographics_df['Percent of Players'] = gender_demographics_df['Percent of Players'].map('{:.0%}'.format)
gender_demographics_df

,Total Players,Percent of Players
Gender,,
Female,81,14%
Male,484,84%
Other/Non-Disclosed,11,2%


In [11]:
#Gender Purchasing Analysis

gender_group_df = purchase_data_df.groupby('Gender')
summarized_gender_df = gender_group_df['Price'].agg([np.count_nonzero,np.sum, np.mean])
summarized_gender_df = summarized_gender_df.rename(columns = {'count_nonzero' : 'Purchase Count', 
                                                      'sum' : 'Total Purchase Value', 'mean' : 'Average Purchase Price'
                                                     })

In [12]:
#Gender Purchasing Analysis
m_avg_tpp = sum(m_df['Price'])/len(m_df["SN"].unique())
nd_avg_tpp = sum(nd_df['Price'])/len(nd_df['SN'].unique())
f_avg_tpp = sum(f_df['Price'])/len(f_df['SN'].unique())

gp_summary_dict = {
    'Gender': ['Female','Male','Other / Non-Disclosed'],
    'Average Purchase Total per Person by Gender': [f_avg_tpp, m_avg_tpp, nd_avg_tpp],
    }

avg_tpp_df = pd.DataFrame(gp_summary_dict)

In [13]:
#Merge Dataframes
gp_merge_df = pd.merge(summarized_gender_df, avg_tpp_df, on='Gender', how="outer")

In [14]:
#Format Dataframe as needed
gp_merge_df.set_index('Gender', inplace = True)
gp_merge_df['Purchase Count'] = gp_merge_df['Purchase Count'].map('{:.0f}'.format)
gp_merge_df['Total Purchase Value'] = gp_merge_df['Total Purchase Value'].map('${:.2f}'.format)
gp_merge_df['Average Purchase Price'] = gp_merge_df['Average Purchase Price'].map('${:.2f}'.format)
gp_merge_df['Average Purchase Total per Person by Gender'] = gp_merge_df['Average Purchase Total per Person by Gender'].map('${:.2f}'.format)

#Print Results
gp_merge_df

,Purchase Count,Total Purchase Value,Average Purchase Price,Average Purchase Total per Person by Gender
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,$1967.64,$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [15]:
#Age Demograpgics - Total Purchases Per SN
purchase_per_SN = purchase_data_df["SN"].value_counts().to_frame()
purchase_per_SN ['index_col'] = purchase_per_SN.index
purchase_per_SN = purchase_per_SN.rename(columns = {'SN' : 'Total Purchases', 
                                                      'index_col' : 'SN',
                                                     })

In [16]:
#Age Demograpgics - Create Bins for Age Groups
bins = [6, 9.9, 13.9, 17.9,21.9,25.9,29.9,33.9,37.9,41.9,46]
age_groups = ['6-10','10-14','14-18','18-22','22-26','26-30','30-34','34-38','38-42','42-46']

purchase_data_df["Age Group"] = pd.cut(purchase_data_df['Age'],bins, labels = age_groups, include_lowest = True)

In [17]:
age_demo_merge_df = pd.merge(purchase_per_SN, purchase_data_df, on = 'SN', how='outer')

In [18]:
#Group by  Age Groups and run aggregate functions
age_group_df = age_demo_merge_df.groupby('Age Group')
agg_df = age_group_df['Price'].agg([np.count_nonzero, np.sum, np.mean])

In [19]:
#Calculate Total Number of Unique Players in Each Age Group
total_players_df = age_demo_merge_df.drop_duplicates(subset='SN')
total_df = total_players_df.groupby('Age Group')
total_per_age = total_df['SN'].agg([np.count_nonzero])

In [20]:
age_demo = pd.merge(agg_df, total_per_age, on = 'Age Group', how='outer')

In [21]:
#Rename Age Group Columns
age_demo = age_demo.rename(columns = {'count_nonzero_y' : 'Total Players',
                                      'count_nonzero_x' : 'Total Purchases',
                                                      'sum' : 'Total Purchase Value',
                                                      'mean' : 'Average Purchase Price',
                                                     })
age_demo = age_demo[['Total Players', 'Total Purchases','Total Purchase Value', 'Average Purchase Price']]

In [22]:
#Caculate Average Purchase Total per Person by Age Group
age_demo['Average Purchase Total per Person by Age Group'] = age_demo['Total Purchase Value']/age_demo['Total Players']

In [23]:
#Format Age Group colmuns as needed
age_demo['Total Purchases'] = age_demo['Total Purchases'].map('{:.0f}'.format)
age_demo['Total Purchase Value'] = age_demo['Total Purchase Value'].map('${:.2f}'.format)
age_demo['Average Purchase Price'] = age_demo['Average Purchase Price'].map('${:.2f}'.format)
age_demo['Average Purchase Total per Person by Age Group'] = age_demo['Average Purchase Total per Person by Age Group'].map('${:.2f}'.format)

#Print Results
age_demo

,Total Players,Total Purchases,Total Purchase Value,Average Purchase Price,Average Purchase Total per Person by Age Group
Age Group,,,,,
6-10,17,23,$77.13,$3.35,$4.54
10-14,20,26,$75.87,$2.92,$3.79
14-18,71,89,$267.60,$3.01,$3.77
18-22,150,210,$647.26,$3.08,$4.32
22-26,189,263,$800.90,$3.05,$4.24
26-30,34,42,$111.10,$2.65,$3.27
30-34,45,64,$191.87,$3.00,$4.26
34-38,27,35,$112.33,$3.21,$4.16
38-42,18,23,$80.72,$3.51,$4.48


In [25]:
#Top Spenders - Group Dataset and run aggregate fuctions
spending_group_df = purchase_data_df.groupby('SN')
summary_sg_df = spending_group_df['Price'].agg([np.count_nonzero, np.sum, np.mean])
summary_sg_df = summary_sg_df.rename(columns = {'count_nonzero' : 'Purchase Count', 
                                                      'sum' : 'Total Purchase Value',
                                                      'mean' : 'Average Purchase Price',
                                                     })

In [26]:
#Top Spenders - Filter Dataframe to find top spenders
top_spenders_df = summary_sg_df.sort_values('Total Purchase Value', ascending=False)
top_spenders_df.head()

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5.0,18.96,3.792000
Idastidru52,4.0,15.45,3.862500
Chamjask73,3.0,13.83,4.610000
Iral74,4.0,13.62,3.405000
Iskadarya95,3.0,13.10,4.366667


In [27]:
#Top Spenders - Format Dataframe and Print Top Spenders
top_spenders_df['Average Purchase Price'] = top_spenders_df['Average Purchase Price'].map('${:.2f}'.format)
top_spenders_df['Total Purchase Value'] = top_spenders_df['Total Purchase Value'].map('${:.2f}'.format)
top_spenders_df['Purchase Count'] = top_spenders_df['Purchase Count'].map('{:.0f}'.format)
top_spenders_df.head()

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


In [28]:
#Most Popular Items - Create data frame with just items
item_list_df = purchase_data_df[['Item ID', 'Item Name', 'Price']]

In [29]:
#Most Popular Items - Create Groupby and do Aggregate Fuctions
item_group_df = item_list_df.groupby('Item Name')
item_agg_df = item_group_df['Price'].agg([np.count_nonzero, np.sum])
item_agg_df = item_agg_df.rename(columns = {'count_nonzero' : 'Purchase Count', 
                                                      'sum' : 'Total Purchase Value',
                                                     })

In [30]:
#Most Popular Items

#Merge Datasets
merge_df = pd.merge(item_agg_df, item_list_df, on="Item Name", how="outer")
merge_df = merge_df.drop_duplicates(subset='Item Name')

#Filter Dataset
popular_items = merge_df.sort_values('Purchase Count', ascending=False)
popular_items = popular_items.reset_index(drop=True)

#Format Dataset
popular_items = popular_items[["Item Name","Item ID","Purchase Count","Price", "Total Purchase Value"]]
popular_items['Total Purchase Value'] = popular_items['Total Purchase Value'].map('${:.2f}'.format)
popular_items['Price'] = popular_items['Price'].map('${:.2f}'.format)
popular_items['Purchase Count'] = popular_items['Purchase Count'].map('{:.0f}'.format)

#Print 5 Most Purchased Items
popular_items.head()

,Item Name,Item ID,Purchase Count,Price,Total Purchase Value
0,Final Critic,92,13,$4.88,$59.99
1,"Oathbreaker, Last Hope of the Breaking Storm",178,12,$4.23,$50.76
2,Persuasion,132,9,$3.19,$28.99
3,Nirvana,82,9,$4.90,$44.10
4,"Extraction, Quickblade Of Trembling Hands",108,9,$3.53,$31.77


In [37]:
#Most Profitable Items

grouped_items_df = item_list_df.groupby('Item Name')
items_profit_df = grouped_items_df['Price'].agg([np.count_nonzero, np.sum])
items_profit_df = items_profit_df.rename(columns = {'count_nonzero' : 'Purchase Count', 
                                                      'sum' : 'Total Purchase Value',
                                                     })

#Merge Datasets
p_merge_df = pd.merge(items_profit_df, item_list_df, on="Item Name", how="outer")
p_merge_df = p_merge_df.drop_duplicates(subset='Item Name')

#Filter Dataset
profit_items = p_merge_df.sort_values('Total Purchase Value', ascending=False)
profit_items = profit_items.reset_index(drop=True)

#Format Dataset
profit_items = profit_items[["Item Name","Item ID","Purchase Count","Price", "Total Purchase Value"]]
profit_items['Total Purchase Value'] = profit_items['Total Purchase Value'].map('${:.2f}'.format)
profit_items['Price'] = profit_items['Price'].map('${:.2f}'.format)
profit_items['Purchase Count'] = profit_items['Purchase Count'].map('{:.0f}'.format)

#Print 5 Most Purchased Items
profit_items.head()

,Item Name,Item ID,Purchase Count,Price,Total Purchase Value
0,Final Critic,92,13,$4.88,$59.99
1,"Oathbreaker, Last Hope of the Breaking Storm",178,12,$4.23,$50.76
2,Nirvana,82,9,$4.90,$44.10
3,Fiery Glass Crusader,145,9,$4.58,$41.22
4,Singed Scalpel,103,8,$4.35,$34.80


# Heroes of Pymoli Trends

1) The majority of the player base comes from the 18-22 and 22-26 age groups. They account for 61% of the revenue and 59% of players.

2) The player base is predominantly male but the average spending was close amongst the genders. With female players spending the most per person on average.

3) The item "Final Critic" had prices of 4.88 and 4.19 possibly leading to it being the most popular and most profitable item in the game.
